<a href="https://colab.research.google.com/github/khoubaib30/dbmsproject/blob/main/Etlprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


convertion of presence sheet to json

In [ ]:
import pandas as pd

# Load Presence data
presence_data = pd.read_excel('/content/drive/MyDrive/presence.xlsx')  # Assuming your Excel file is named 'presence.xlsx'

# Convert to JSON and save
presence_data.to_json('presencejson.json', orient='records', date_format='iso')

conversion of absence sheet to json

In [ ]:
import pandas as pd

# Load Presence data
presence_data = pd.read_excel('/content/drive/MyDrive/absence.xlsx')  # Assuming your Excel file is named 'presence.xlsx'

# Convert to JSON and save
presence_data.to_json('absencejson.json', orient='records', date_format='iso')

conversion of congee sheet to json

In [ ]:
import pandas as pd

# Load Presence data
presence_data = pd.read_excel('/content/drive/MyDrive/congees.xlsx')  # Assuming your Excel file is named 'presence.xlsx'

# Convert to JSON and save
presence_data.to_json('congeejson.json', orient='records', date_format='iso')

unify the names in all databases zeyda ()


In [ ]:
!pip install rapidfuzz

import pandas as pd
from rapidfuzz import process

# Load JSON files into pandas DataFrames
employees_df = pd.read_json('/content/drive/MyDrive/employeesjson.json')
absence_df = pd.read_json('/content/drive/MyDrive/absencejson.json')
presence_df = pd.read_json('/content/drive/MyDrive/presencejson.json')
congee_df = pd.read_json('/content/drive/MyDrive/congeejson.json')

# Preprocess the data (lowercase, remove special characters)
employees_df['preprocessed_name'] = employees_df['prenom'].str.lower().str.replace('[^a-zA-Z0-9]', '')

absence_df['preprocessed_name'] = absence_df['prenom'].str.lower().str.replace('[^a-zA-Z0-9]', '')
presence_df['preprocessed_name'] = presence_df['prenom'].str.lower().str.replace('[^a-zA-Z0-9]', '')
congee_df['preprocessed_name'] = congee_df['prenom'].str.lower().str.replace('[^a-zA-Z0-9]', '')

# Function to find the closest match using rapidfuzz
def find_closest_match(name, reference_names):
    result = process.extractOne(name, reference_names)
    return result

# Initialize lists to store unmatched names and matches for inspection
unmatched_names = []
matches = []

# Map names and collect unmatched names and matches
for df in [absence_df, presence_df, congee_df]:
    df['match_result'] = df['preprocessed_name'].apply(lambda x: find_closest_match(x, employees_df['preprocessed_name']))
    df['mapped_name'] = df['match_result'].apply(lambda x: x[0] if x[1] >= 70 else None)  # Adjust threshold as needed
    unmatched_names.extend(df[df['mapped_name'].isnull()]['prenom'].tolist())
    matches.extend(df[df['mapped_name'].notnull()][['prenom', 'mapped_name', 'match_result']].values.tolist())

# Print unmatched names
print("Unmatched Names:")
print(unmatched_names)

# Print matches for inspection
print("\nMatches for Inspection:")
for match in matches:
    print(f"Original: {match[0]}, Mapped: {match[1]}, Score: {match[2][1]}")

# Update names in absence_df, presence_df, and congee_df using mapped names
for df in [absence_df, presence_df, congee_df]:
    df['prenom'] = df['mapped_name'].apply(lambda x: employees_df.loc[employees_df['preprocessed_name'] == x, 'prenom'].iloc[0] if x else None)

# Drop unnecessary columns
for df in [absence_df, presence_df, congee_df]:
    df.drop(['preprocessed_name', 'mapped_name', 'match_result'], axis=1, inplace=True)

# Save the updated DataFrames to new JSON files without changing date and time formats
absence_df.to_json('absenceunified.json', orient='records')
presence_df.to_json('presenceunified.json', orient='records')
congee_df.to_json('congeeunified.json', orient='records')

standardize the date and time fields, you can convert all date fields to the format YYYY-MM-DD and time fields to the format HH:MM:SS.

In [ ]:
# Importing necessary library
import pandas as pd

# Function to standardize date format to YYYY-MM-DD
def standardize_date_format(date_series):
    return pd.to_datetime(date_series).dt.strftime('%Y-%m-%d')

# Function to standardize time format to HH:MM:SS (with seconds as "00")
def standardize_time_format(time_series):
    return pd.to_datetime(time_series, format='%H:%M', errors='coerce').dt.strftime('%H:%M:%S')

# Load the JSON files into pandas DataFrames
absence_df = pd.read_json('/content/drive/MyDrive/absenceunified.json')
congee_df = pd.read_json('/content/drive/MyDrive/congeeunified.json')
presence_df = pd.read_json('/content/drive/MyDrive/presenceunified.json')

# Standardizing date fields to YYYY-MM-DD format
absence_df['Date'] = standardize_date_format(absence_df['Date'])
congee_df['DATE DEBUT'] = standardize_date_format(congee_df['DATE DEBUT'])
congee_df['REPRISE'] = standardize_date_format(congee_df['REPRISE'])
presence_df['Date'] = standardize_date_format(presence_df['Date'])

# Define the time fields for each DataFrame
absence_time_fields = ['Enregistrement', 'Depart', 'Pointer entree', 'Pointer sortie']
presence_time_fields = ['Premier pointage', 'Dernier pointage']

# Standardizing time fields in absence_df and presence_df
for field in absence_time_fields:
    if field in absence_df.columns:
        absence_df[field] = standardize_time_format(absence_df[field])

for field in presence_time_fields:
    if field in presence_df.columns:
        presence_df[field] = standardize_time_format(presence_df[field])

absence_df.to_json('absencedate.json', orient='records', date_format='iso')
congee_df.to_json('congeedate.json', orient='records', date_format='iso')
presence_df.to_json('presencedate.json', orient='records', date_format='iso')

assign a unique id to each departement for better analysis

In [ ]:
import pandas as pd
import json

# Load data from the "employees" JSON file into a Pandas DataFrame
df_employees = pd.read_json('/content/drive/MyDrive/employeesjson.json')

# Extract unique departments and assign unique IDs
unique_departments = df_employees['departement'].unique()
department_id_mapping = {department: idx + 1 for idx, department in enumerate(unique_departments)}

# Add department IDs to the original file
for index, row in df_employees.iterrows():
    df_employees.at[index, 'department_id'] = department_id_mapping[row['departement']]

# Save the updated data back to the original file
df_employees.to_json('/content/drive/MyDrive/employeesfinal.json', orient='records')

# Create a new JSON file with department IDs
department_id_json = [{'department_id': department_id, 'department': department} for department, department_id in department_id_mapping.items()]
with open('/content/drive/MyDrive/department_id.json', 'w') as json_file:
    json.dump(department_id_json, json_file, indent=2)



count of emplyees per department

In [ ]:
import pandas as pd
import json

# Load the department_id JSON file
with open('/content/drive/MyDrive/department_id.json', 'r') as file:
    department_id_data = json.load(file)

# Load the employeesfinal JSON file
df_employees = pd.read_json('/content/drive/MyDrive/employeesfinal.json')

# Create a mapping from department names to department IDs
department_name_to_id = {item['department']: item['department_id'] for item in department_id_data}

# Count the number of employees per department name
department_name_count = df_employees['departement'].value_counts().to_dict()

# Update the department_id_data with the number of employees per department ID
for item in department_id_data:
    department_name = item['department']
    department_id = item['department_id']
    item['number_of_employees'] = department_name_count.get(department_name, 0)

# Save the updated department_id_data back to JSON
with open('updated_department_id.json', 'w') as file:
    json.dump(department_id_data, file, indent=2)

include the ID,department and department_id in congee file


In [ ]:
import json

# Load the data from the files
with open('/content/drive/MyDrive/congeedate.json', 'r') as file:
    finalcongee_data = json.load(file)

with open('/content/drive/MyDrive/employeesfinal.json', 'r') as file:
    employees_data = json.load(file)

# Create a mapping of 'prenom' to 'id', 'departement', and 'department_id' from the employees data
employee_dept_mapping = {employee['prenom']: {'id': employee['ID'], 'departement': employee['departement'], 'department_id': employee['department_id']}
                         for employee in employees_data}

# Update the finalcongee data with the department information and employee ID
for record in finalcongee_data:
    prenom = record['prenom']
    if prenom in employee_dept_mapping:
        record.update(employee_dept_mapping[prenom])

# Save the updated finalcongee data to a new JSON file
with open('updated_congee.json', 'w') as file:
    json.dump(finalcongee_data, file)

include department_id in presence and absence files

In [ ]:
import pandas as pd
import json

def load_json_file(file_path):
    with open(file_path, 'r') as file:
        return pd.DataFrame(json.load(file))

# Load the JSON files
department_id_df = load_json_file('/content/drive/MyDrive/department_id.json')
absence_df = load_json_file('/content/drive/MyDrive/absencedate.json')
presence_df = load_json_file('/content/drive/MyDrive/presencedate.json')

# Merge absence_df with department_id_df to get 'department_id' based on 'departement'
absence_df = pd.merge(absence_df, department_id_df, left_on='departement', right_on='department', how='left')

# Merge presence_df with department_id_df to get 'department_id' based on 'departement'
presence_df = pd.merge(presence_df, department_id_df, left_on='departement', right_on='department', how='left')

# Drop redundant 'department' column after the merge
absence_df.drop('department', axis=1, inplace=True)
presence_df.drop('department', axis=1, inplace=True)

# Save the updated DataFrames to new JSON files without altering date and time formats

absence_df.to_json('absence_updated.json', orient='records', date_format='iso')
presence_df.to_json('presence_updated.json', orient='records', date_format='iso')

update of "Premier pointage" and "dernier pointage" fields and cration of columns of total working time and supplemantay working time


In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Load the JSON file
df = pd.read_json('/content/drive/MyDrive/presence_updated.json')

# Function to convert time to datetime.time object
def convert_to_time(time_str):
    if pd.isna(time_str):
        return None
    return datetime.strptime(time_str, '%H:%M:%S').time()

# Function to calculate working hours
def calculate_working_hours(start, end):
    if start is None or end is None:
        return 0
    start_dt = datetime.combine(datetime.today(), start)
    end_dt = datetime.combine(datetime.today(), end)
    if start_dt > end_dt:
        end_dt += timedelta(days=1)
    return round((end_dt - start_dt).seconds / 3600, 1)

# Updating "Premier pointage" and "Dernier pointage" columns
df['Premier pointage'] = df['Premier pointage'].apply(convert_to_time)
df['Dernier pointage'] = df['Dernier pointage'].apply(convert_to_time)

for index, row in df.iterrows():
    if row['Jour de semaine'] not in ['Samedi', 'Dimanche']:
        if row['Premier pointage'] is not None and row['Dernier pointage'] is not None:
            if row['Premier pointage'] == row['Dernier pointage']:
                if row['Premier pointage'] < datetime.strptime('12:00:00', '%H:%M:%S').time():
                    df.at[index, 'Dernier pointage'] = datetime.strptime('17:30:00', '%H:%M:%S').time()
                elif row['Premier pointage'] > datetime.strptime('12:00:00', '%H:%M:%S').time():
                    df.at[index, 'Premier pointage'] = datetime.strptime('08:30:00', '%H:%M:%S').time()

# Calculating total working time
df['Total working time'] = df.apply(lambda row: calculate_working_hours(row['Premier pointage'], row['Dernier pointage']), axis=1)

# Calculating supplementary working time
df['Supplementary working time'] = df.apply(lambda row: max(0, row['Total working time'] - 9) if row['Jour de semaine'] not in ['Samedi', 'Dimanche'] else 0, axis=1)

# Converting Date column to string in yyyy-mm-dd format
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

# Saving the updated presence file
df.to_json('final_presence.json', orient='records', date_format='iso')


create a field totale congee in the congee.json file

In [ ]:
import pandas as pd

# Load Absence data from JSON
absence_data = pd.read_json('/content/drive/MyDrive/updated_congee.json', orient='records')

# Convert 'REPRISE' and 'DATE DEBUT' to datetime
absence_data['REPRISE'] = pd.to_datetime(absence_data['REPRISE'], errors='coerce')
absence_data['DATE DEBUT'] = pd.to_datetime(absence_data['DATE DEBUT'])

# Calculate 'Total Congee' column
absence_data['Total Congee'] = (absence_data['REPRISE'] - absence_data['DATE DEBUT']).dt.total_seconds() / (24 * 3600)

# Adjust 'Total Congee' for cases where 'REPRISE' is empty or 'REMARQUE' contains "1/2 JR"
absence_data.loc[absence_data['REPRISE'].isnull() | (absence_data['REMARQUE'] == "1/2 JR"), 'Total Congee'] = 0.5

# Format the date columns as 'yyyy-mm-dd' strings
absence_data['REPRISE'] = absence_data['REPRISE'].dt.strftime('%Y-%m-%d')
absence_data['DATE DEBUT'] = absence_data['DATE DEBUT'].dt.strftime('%Y-%m-%d')

# Print the updated Absence data for debugging
print("\nUpdated Absence Data:")
print(absence_data[['REPRISE', 'DATE DEBUT', 'REMARQUE', 'Total Congee']])

# Save the updated Absence data to a new JSON file with 'yyyy-mm-dd' format
absence_data.to_json('finalcongee.json', orient='records', date_format='iso', default_handler=str)

create the fact table

In [ ]:
import pandas as pd

# Load the data from each file
presence_data = pd.read_json('/content/drive/MyDrive/final_presence.json')
absence_data = pd.read_json('/content/drive/MyDrive/finalabsence.json')
congee_data = pd.read_json('/content/drive/MyDrive/finalcongee.json')

# Process the presence data
presence_data['Weekend'] = presence_data['Jour de semaine'].isin(['Samedi', 'Dimanche']).astype(int)
presence_data['Present'] = 1 - presence_data['Weekend']
presence_data['Congee'] = 0
presence_data['Absent'] = 0
presence_data['Total_Working_Time'] = presence_data['Total working time']
presence_data['Supplementary_Working_Time'] = presence_data['Supplementary working time']
presence_data_processed = presence_data[['department_id', 'ID', 'Date', 'Total_Working_Time', 'Supplementary_Working_Time', 'Absent', 'Present', 'Weekend', 'Congee']]
presence_data_processed = presence_data_processed.rename(columns={'department_id': 'Department_id', 'ID': 'Employee_id'})

# Process the absence data
absence_data['Absent'] = 1
absence_data['Present'] = 0
absence_data['Weekend'] = 0
absence_data['Congee'] = 0
absence_data['Total_Working_Time'] = 0
absence_data['Supplementary_Working_Time'] = 0
absence_data_processed = absence_data[['department_id', 'ID', 'Date', 'Total_Working_Time', 'Supplementary_Working_Time', 'Absent', 'Present', 'Weekend', 'Congee']]
absence_data_processed = absence_data_processed.rename(columns={'department_id': 'Department_id', 'ID': 'Employee_id'})

# Process the congee data
congee_data_processed = pd.DataFrame()
for index, row in congee_data.iterrows():
    if row['Total Congee'] == 0.5:
        # Create a single-day entry with congee 0.5
        congee_entry = pd.DataFrame({
            'Department_id': [row['department_id']],
            'Employee_id': [row['id']],
            'Date': [row['DATE DEBUT']],
            'Total_Working_Time': [0],
            'Supplementary_Working_Time': [0],
            'Absent': [0],
            'Present': [0],
            'Weekend': [0],
            'Congee': [0.5]
        })
    else:
        # Create entries for each date between 'DATE DEBUT' and 'REPRISE'
        date_range = pd.date_range(start=row['DATE DEBUT'], end=row['REPRISE'], closed='left')
        congee_entry = pd.DataFrame({
            'Department_id': row['department_id'],
            'Employee_id': row['id'],
            'Date': date_range,
            'Total_Working_Time': 0,
            'Supplementary_Working_Time': 0,
            'Absent': 0,
            'Present': 0,
            'Weekend': 0,
            'Congee': 1
        })
    congee_data_processed = pd.concat([congee_data_processed, congee_entry], ignore_index=True)

# Combine all processed data
combined_data = pd.concat([presence_data_processed, absence_data_processed, congee_data_processed], ignore_index=True)

# Drop rows with missing values in Department_id or Employee_id
combined_data_cleaned = combined_data.dropna(subset=['Department_id', 'Employee_id'])

# Convert Department_id and Employee_id to integer type
combined_data_cleaned['Department_id'] = combined_data_cleaned['Department_id'].astype(int)
combined_data_cleaned['Employee_id'] = combined_data_cleaned['Employee_id'].astype(int)

# Round float columns to the first decimal
combined_data_cleaned['Total_Working_Time'] = combined_data_cleaned['Total_Working_Time'].round(1)
combined_data_cleaned['Supplementary_Working_Time'] = combined_data_cleaned['Supplementary_Working_Time'].round(1)
combined_data_cleaned['Congee'] = combined_data_cleaned['Congee'].round(1)

# Ensure that the 'Date' column is in datetime format and sort by Date in ascending order
combined_data_sorted = combined_data_cleaned.copy()
combined_data_sorted['Date'] = pd.to_datetime(combined_data_sorted['Date'])
combined_data_sorted = combined_data_sorted.sort_values(by='Date')
# Save the final output to a CSV file
combined_data_sorted.to_csv('/content/drive/MyDrive/fact1.csv', index=False)


create dimensions

In [ ]:
import pandas as pd
import json
from datetime import datetime

# Load the data from the files
# Replace these lines with your actual file loading code
with open('/content/drive/MyDrive/final_presence.json', 'r') as file:
    presence_data = json.load(file)

with open('/content/drive/MyDrive/finalabsence.json', 'r') as file:
    absence_data = json.load(file)

with open('/content/drive/MyDrive/finalcongee.json', 'r') as file:
    congee_data = json.load(file)

with open('/content/drive/MyDrive/employeesfinal.json', 'r') as file:
    employee_data = json.load(file)

with open('/content/drive/MyDrive/updated_department_id.json', 'r') as file:
    department_data = json.load(file)

# Convert the data to pandas DataFrames
df_presence = pd.DataFrame(presence_data)
df_absence = pd.DataFrame(absence_data)
df_congee = pd.DataFrame(congee_data)
df_employee = pd.DataFrame(employee_data)
df_department = pd.DataFrame(department_data)


# Convert 'Date' to datetime in presence, absence, and congee data
df_presence['Date'] = pd.to_datetime(df_presence['Date'])
df_absence['Date'] = pd.to_datetime(df_absence['Date'])
df_congee['DATE DEBUT'] = pd.to_datetime(df_congee['DATE DEBUT'])
df_congee['REPRISE'] = pd.to_datetime(df_congee['REPRISE'])

# Create the Date dimension
date_dim = pd.DataFrame(df_presence['Date'].unique(), columns=['Date'])
date_dim['Date_id'] = date_dim.index + 1  # Assuming Date_id starts from 1
date_dim['Year'] = date_dim['Date'].dt.year
date_dim['Month'] = date_dim['Date'].dt.month
date_dim['Day'] = date_dim['Date'].dt.day
date_dim['Date'] = date_dim['Date'].dt.strftime('%Y-%m-%d')  # Format date as 'yyyy-mm-dd'

# Create the Employee dimension with Department_id
employee_dim = df_employee.rename(columns={'id': 'Employee_id', 'prenom': 'Name', 'departement': 'Department', 'department_id': 'Department_id'})
# Drop the department name column from the employee dimension DataFrame
employee_dim.drop(columns=['Department'], inplace=True)
employee_dim['Department_id'] = employee_dim['Department_id'].astype(int)

# Create the department dimension
department_dim = df_department.rename(columns={'department': 'Name', 'department_id': 'Department_id', 'employee_count': 'Number_of_employees'})
department_dim['Department_id'] = department_dim['Department_id'].astype(int)

date_dim.to_csv('date_dimension.csv', index=False)
employee_dim.to_csv('employee_dimension.csv', index=False)
department_dim.to_csv('department_dimension.csv', index=False)

include Date_id in the fact table

In [ ]:
# Load the combined employee data and the date dimension data
combined_employee_data = pd.read_csv('/content/drive/MyDrive/fact1.csv')
date_dimension_data = pd.read_csv('/content/drive/MyDrive/date_dimension.csv')

# Convert 'Date' columns to datetime format for merging
combined_employee_data['Date'] = pd.to_datetime(combined_employee_data['Date'])
date_dimension_data['Date'] = pd.to_datetime(date_dimension_data['Date'])

# Merge the combined employee data with the date dimension data to include the Date_id column
combined_data_with_date_id = combined_employee_data.merge(date_dimension_data[['Date', 'Date_id']], on='Date', how='left')

# Drop the 'Date' column from the merged data
combined_data_with_date_id.drop('Date', axis=1, inplace=True)

# Save the updated combined data to a new CSV file
combined_data_with_date_id.to_csv('/content/drive/MyDrive/hrfacttable.csv', index=False)

create record_id column

In [ ]:
# Load the combined employee data and the date dimension data
facttab = pd.read_csv('/content/drive/MyDrive/fact2.csv')

# Add a 'record_id' column to the data, starting at 1 and incrementing for each row
facttab['record_id'] = range(1, len(facttab) + 1)

# Save the data with 'record_id' column to a new CSV file
facttab.to_csv('hr_fact_table.csv', index=False)
